In [2]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1,remove=('headers', 'footers', 'quotes'))

In [3]:
n_samples = 2000
documents = dataset.data[:n_samples]
print(documents[20])
print(len(documents))





	I'd like to see this info as well.  As for wavelength, I think
you're primarily going to find two - 880 nM +/- a bit, and/or 950 nM
+/- a bit.  Usually it is about 10 nM either way.  The two most common
I have seen were 880 and 950 but I have also heard of 890 and 940.
I'm not sure that the 10 nM one way or another will make a great deal of
difference.

	Another suggestion - find a brand of TV that uses an IR remote,
and go look at the SAMS photofact for it.  You can often find some very
detailed schematics and parts list for not only the receiver but the
transmitter as well, including carrier freq. specs. and tone decoding
specs. if the system uses that.
2000


In [4]:
def to_seq(documents):
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()

    # Text data remove empty item
    text_data = [doc for doc in documents if doc != '']

    # Convert text data into a list of comments after stop words and stemming are accounted for
    for idx, doc in enumerate(text_data):

        # Convert comment all to lower case
        raw_lower = doc.lower()

        # Tokenize comment
        doc_token = tokenizer.tokenize(raw_lower)

        # Only keep letters  and must be more than two character
        word_token = [re.sub(r'[^a-zA-Z]','', word) and re.sub(r'\b\w{,1}\b', '', word) for word in doc_token]

        # Take out stop words
        no_stop_token = [word for word in word_token if not word in eng_stop if word != '']

        # stem words
        stemed_token = [str(p_stemmer.stem(word)) for word in no_stop_token]

        # Replace comment with cleaned list of words
        text_data[idx] = no_stop_token
        
    return text_data

In [5]:
# data preprocessing
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from __future__ import division

# Set seed so we get same random allocation on each run of code
np.random.seed(2017)

# Load tokenizer function
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Tokenizing an item for observation
tokenize_obs = tokenizer.tokenize(documents[20])
print('Tokenize 21st result: \n%s' % tokenize_obs)

eng_stop = stopwords.words('english')
# Print a few stop words
print('Stop words in english: \n%s' % eng_stop[1:10])

text_data = to_seq(documents)

print ("The 21st comment (after processing) is: \n%s" % text_data[20])
# Need to convert our list of list data into single list if words is not ''
words_list = [words for doc_word_list in text_data for words in doc_word_list]
# Vocabulary is the set of unique words used
vocab_total = set(words_list)

print('Few words from vocabulary list: \n%s' % list(vocab_total)[1:7])
print('Number of unique words in data: %s' % len(vocab_total))

print("Orginal total documents: ", str(n_samples), ". After process total document: ", str(len(text_data)))

# Convert each comment into a vector by replacing the words by their unique ID
text_ID = []

# Loop over cleaned text data
for doc_word_list in text_data:
    # Append doc replaced by unique word IDs
    comment_vector = [list(vocab_total).index(words) for words in doc_word_list]
    text_ID.append(comment_vector)

# Let's check the first comment
print('21st comment as a vector of word IDs is: \n%s' % text_ID[20])

Tokenize 21st result: 
['I', 'd', 'like', 'to', 'see', 'this', 'info', 'as', 'well', 'As', 'for', 'wavelength', 'I', 'think', 'you', 're', 'primarily', 'going', 'to', 'find', 'two', '880', 'nM', 'a', 'bit', 'and', 'or', '950', 'nM', 'a', 'bit', 'Usually', 'it', 'is', 'about', '10', 'nM', 'either', 'way', 'The', 'two', 'most', 'common', 'I', 'have', 'seen', 'were', '880', 'and', '950', 'but', 'I', 'have', 'also', 'heard', 'of', '890', 'and', '940', 'I', 'm', 'not', 'sure', 'that', 'the', '10', 'nM', 'one', 'way', 'or', 'another', 'will', 'make', 'a', 'great', 'deal', 'of', 'difference', 'Another', 'suggestion', 'find', 'a', 'brand', 'of', 'TV', 'that', 'uses', 'an', 'IR', 'remote', 'and', 'go', 'look', 'at', 'the', 'SAMS', 'photofact', 'for', 'it', 'You', 'can', 'often', 'find', 'some', 'very', 'detailed', 'schematics', 'and', 'parts', 'list', 'for', 'not', 'only', 'the', 'receiver', 'but', 'the', 'transmitter', 'as', 'well', 'including', 'carrier', 'freq', 'specs', 'and', 'tone', 'deco

In [2]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [6]:
# Initialize hyperparameters in LDA

# Dirichlet parameters
# Alpha is the parameter for the prior topic distribution within documents
alpha = 0.2

# Beta is the parameter for the prior topic distribution within documents
beta = 0.001

# Text corpus itterations
corpus_itter = 200

# Number of topics
K = 2

# Vocabulary size
V = len(vocab_total)

# Number of Documents
D = len(text_ID)

# For practical implementation, we will generate the following three count matrices:
# 1) Word-Topic count matrix, 2) Topic-Document assignment matrix, 3) Document-Topic count matrix

# Initialize word-topic count matrix (size K x V, K = # topics, V = # vocabulary)
word_topic_count = np.zeros((K,V))

# Initialize topic-document assignment matrix
topic_doc_assign = [np.zeros(len(doc_word_list)) for doc_word_list in text_ID] 

# Initialize document-topic matrix
doc_topic_count = np.zeros((D,K))

# Generate word-topic count matrix with randomly assigned topics
# Loop over documents
for doc in range(D):
    
    # Loop over words in given document
    for word in range(len(text_ID[doc])):

        # Step 1: Randomly assign topics to each word in document
        # Note random.choice generates number {0,...,K-1}
        topic_doc_assign[doc][word] = np.random.choice(K,1)
        
        # Record word-topic and word-ID
        word_topic = int(topic_doc_assign[doc][word])
        word_doc_ID = text_ID[doc][word]
        
        # Step 2: Generate word-topic count matrix with randomly assigned topics
        word_topic_count[word_topic][word_doc_ID] += 1
# Print word-topic matrix
print('Word-topic count matrix with random topic assignment: \n%s' % word_topic_count)

# Step 3: Generate document-topic count matrix with randomly assigned topics
# Loop over documents (D = numb. docs)
for doc in range(D):
    
    # Loop over topics (K = numb. topics)
    for topic in range(K):
        
        # topic-document vector
        topic_doc_vector = topic_doc_assign[doc]
        
        # Update document-topic count
        doc_topic_count[doc][topic] = sum(topic_doc_vector == topic)
        
# Print document-topic matrix
print('Subset of document-topic count matrix with random topic assignment: \n%s' % doc_topic_count[0:5])

Word-topic count matrix with random topic assignment: 
[[1. 1. 3. ... 0. 7. 3.]
 [0. 2. 1. ... 1. 7. 1.]]
Subset of document-topic count matrix with random topic assignment: 
[[36. 31.]
 [28. 26.]
 [26. 38.]
 [46. 36.]
 [22. 12.]]


In [7]:
print(len(topic_doc_assign))

1965


In [ ]:
# Main part of LDA algorithm (takes a few minutes to run)
# Run through text corpus multiple times
for itter in range(corpus_itter):
    
    # Loop over all documents
    for doc in range(D):
        
        # Loop over words in given document
        for word in range(len(text_ID[doc])):
            
            # Initial topic-word assignment
            init_topic_assign = int(topic_doc_assign[doc][word])
            
            # Initial word ID of word 
            word_id = text_ID[doc][word]
            
            # Before finiding posterior probabilities, remove current word from count matrixes
            word_topic_count[init_topic_assign][word_id] -=1
            doc_topic_count[doc][init_topic_assign] -= 1
            
            # Find probability used for reassigning topics to words within documents
            
            # Denominator in first term (Numb. of words in doc + numb. topics * alpha)
            denom1 = sum(doc_topic_count[doc]) + K*alpha
            
            #print(doc_topic_count[doc])
            
            # Denominator in second term (Numb. of words in topic + numb. words in vocab * beta)
            denom2 = np.sum(word_topic_count, axis = 1) + V*beta
            
            #print(word_topic_count)
            #print(np.sum(word_topic_count, axis = 1))
            
            # Numerators, number of words assigned to a topic + prior dirichlet param
            numerator1 = [doc_topic_count[doc][col] for col in range(K)] 
            numerator1 = np.array(numerator1) + alpha
            numerator2 = [word_topic_count[row][word_id] for row in range(K)]
            numerator2 = np.array(numerator2) + beta
            
            # Compute conditional probability of assigning each topic
            # Recall that this is obtained from gibbs sampling
            prob_topics = (numerator1/denom1)*(numerator2/denom2)
            prob_topics = prob_topics/sum(prob_topics)
                                    
            # Update topic assignment (topic can be drawn with prob. found above)
            update_topic_assign = np.random.choice(K,1,list(prob_topics))
            topic_doc_assign[doc][word] = update_topic_assign
            
            # Add in current word back into count matrixes
            doc_topic_count[doc][init_topic_assign] += 1
            word_topic_count[init_topic_assign][word_id] +=1
        #break
    #break

In [ ]:
# Compute posterior mean of document-topic distribution
theta = (doc_topic_count+alpha)
theta_row_sum = np.sum(theta, axis = 1)
theta = theta/theta_row_sum.reshape((D,1))
print(theta.shape)

# Print document-topic mixture
print('Subset of document-topic mixture matrix: \n%s' % theta[0:3])
print (theta[10])
print (documents[10])

# Compute posterior mean of word-topic distribution within documents
phi = (word_topic_count + beta)
phi_row_sum = np.sum(phi, axis = 1)
phi = phi/phi_row_sum.reshape((K,1))
print(phi.shape)
# Print topic-word mixture
print('Topic-word mixture matrix: \n%s' % phi)

# Explore the top words that make up each topic 
# Initialize list of dictionaries
list_dict_topics = []

# Loop over topics
for topic in range(K):
    
    # Initialize (vocab,prob) dictionary
    mydict = {}
    
    # Loop over vocabular
    for word in range(V):
        # Create dictionary {(vocab,prob)}
        mydict[list(vocab_total)[word]] = phi[topic][word]
        
    # Create list of dictionaries
    list_dict_topics.append(mydict)
    
# First topic
# The first 10 words are ignored, because they most overlap with topic 2
# Commonly appearing words in topic 1
sorted([(value,key) for (key,value) in list_dict_topics[0].items()])[::-1][10:30]

# Second topic
# The first 10 words are ignored, because they most overlap with topic 1
# Commonly appearing words in topic 2
sorted([(value,key) for (key,value) in list_dict_topics[1].items()])[::-1][10:30]

In [10]:
print('Subset of document-topic mixture matrix: \n%s' % theta[0:3])
print (theta[10])
print (documents[10])

Subset of document-topic mixture matrix: 
[[0.53709199 0.46290801]
 [0.51838235 0.48161765]
 [0.4068323  0.5931677 ]]
[0.50382507 0.49617493]
Archive-name: graphics/resources-list/part1
Last-modified: 1993/04/17


Computer Graphics Resource Listing : WEEKLY POSTING [ PART 1/3 ]
Last Change : 17 April 1993

Many FAQs, including this Listing, are available on the archive site
pit-manager.mit.edu (alias rtfm.mit.edu) [18.172.1.27] in the directory
pub/usenet/news.answers.  The name under which a FAQ is archived appears
in the Archive-name line at the top of the article.
This FAQ is archived as graphics/resources-list/part[1-3]

There's a mail server on that machine. You send a e-mail message to
mail-server@pit-manager.mit.edu  containing the keyword "help" (without
quotes!) in the message body.

You can see in many other places for this Listing. See the item:

0. Places to find the Resource Listing

for more information.

Items Changed:
--------------

RE-ARRANGED the subjects, in order t

In [ ]:
lda = LatentDirichletAllocation(n_components=2, random_state=2017)